# Vitis Unified backend

### Why Vitis Unified Backend
- For ease of HLS4ML development, we require the simple full flow to build, deploy, and execute the model into XILINX FPGA. The old vitis backend did support only ip generation without cocerning the FPGA IO interface. Moreover, the vivado HLS used by vivado accelerator backend is end of support right now.
- The Vitis based on V++, vitis-run, and vitis_ide should be used

In [1]:
from pathlib import Path

import numpy as np
import os
from tensorflow.keras.layers import Activation, Dense, GlobalAveragePooling1D, Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model, load_model

import hls4ml
import hls4ml.model
import os

2025-08-29 16:01:57.763560: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-29 16:01:57.863701: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### set/get the neccesary variable
- please substitute the ```/tools/Xilinx/Vitis/2023.2``` with your path

In [2]:
test_root_path = Path(os.getcwd())
os.environ['XILINX_VITIS'] = "/tools/Xilinx/Vitis/2023.2"
os.environ['PATH'] = os.environ['XILINX_VITIS'] + '/bin:' + os.environ['PATH']

modelName    = "simpleSkip.keras"
testCaseName = "inputX_1.npy"

#### create value testing function

In [3]:
def checkEqual(a, b):
    equal = np.array_equal(a, b)
    if equal:
        print("Test pass both are equal \U0001F642")
    else:
        print("Test Fail both are not equal \U0001F62C")

#### create testcase gen function

In [4]:
def createSimpleTestcase(inputShape=(4, 4, 1), fileName = "inputX.npy"):
    n_in = np.random.rand(*inputShape).astype(np.float32)
    np.save(test_root_path / fileName, n_in)

#### define simple UNET model function

In [5]:
def createSimpleUnet(input_shape=(4, 4, 1)):
    inputs = Input(input_shape)
    # Encoder
    c1 = Conv2D(2, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)
    # Bottleneck
    bn = Conv2D(4, (3, 3), activation='relu', padding='same')(p1)
    # Decoder
    u1 = UpSampling2D((2, 2))(bn)
    concat1 = Concatenate()([u1, c1])
    c2 = Conv2D(2, (3, 3), activation='relu', padding='same')(concat1)
    # Output layer (1 channel)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c2)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy')
    model.save(test_root_path / modelName)


def loadSimpleUnet():
    model = load_model(test_root_path / modelName)
    return model

#### create prediction function

In [6]:
def bridgeTestProject(model, io_type, granularity, backend, inputNp):
    config = hls4ml.utils.config_from_keras_model(model, granularity=granularity)
    output_dir = str(test_root_path / f"hls4mlprj_bridgeTest_{io_type}_{granularity}_{backend}")
    ############### mono model build
    print("config MONO model")
    hls_model = hls4ml.converters.convert_from_keras_model(
        model, hls_config=config, output_dir=output_dir, backend=backend, io_type=io_type,
        board='zcu102', part='xczu9eg-ffvb1156-2-e', clock_period='10ns'
    )
    hls_model.compile()

    y_pred = hls_model.predict(inputNp)
    print(f"predict from {backend}")
    print(y_pred)
    print(type(y_pred))
    print("-----------------------")

    return y_pred

#### test and compare with old vitis simulation backend

In [7]:
createSimpleUnet()
createSimpleTestcase((1, 4,4,1), testCaseName)

####################################################################
########## bridge test #############################################
####################################################################

input_1 = np.load(test_root_path / testCaseName)

model = loadSimpleUnet()
unifiedPred = bridgeTestProject(model, "io_stream", "name", 'VitisUnified', input_1)
vitisPred   = bridgeTestProject(model, "io_stream", "name", 'Vitis', input_1)

checkEqual(unifiedPred, vitisPred)

config MONO model
predict from VitisUnified
[0.5        0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.5        0.5        0.5        0.48828125
 0.5        0.5        0.5        0.46875   ]
<class 'numpy.ndarray'>
-----------------------
config MONO model
predict from Vitis
[0.5        0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.5        0.5        0.5        0.48828125
 0.5        0.5        0.5        0.46875   ]
<class 'numpy.ndarray'>
-----------------------
Test pass both are equal 🙂


#### create build the model

the XPFM_PATH is the generated platform file. Xilinx provides some default file for some board. but if you require the custom built one. you can follow this tutorial. https://github.com/Xilinx/Vitis-Tutorials/blob/2023.2/Vitis_Platform_Creation/Design_Tutorials/01-Edge-KV260/step1.md

In [8]:
XPFM_PATH = '/tools/Xilinx/Vitis/2023.2/base_platforms/xilinx_zcu102_base_202320_1/xilinx_zcu102_base_202320_1.xpfm'

In [9]:
def createHlsProject(model, io_type, granularity, backend):

    config = hls4ml.utils.config_from_keras_model(model, granularity=granularity)
    output_dir = str(test_root_path / f"hls4mlprjBuildTest_{io_type}_{granularity}_{backend}")
    ############### mono model build
    print("config MONO model")
    hls_model = hls4ml.converters.convert_from_keras_model(
        model, hls_config=config, output_dir=output_dir, backend=backend, io_type=io_type,
        board='zcu102', part='xczu9eg-ffvb1156-2-e', clock_period='10ns',
        in_stream_buf_size = 128,
        out_stream_buf_size = 128,
        xpfmPath = XPFM_PATH
    )
    hls_model.compile()
    hls_model.build(csim=False, synth=True, bitfile = True)

In [10]:
model = loadSimpleUnet()
unifiedPred = createHlsProject(model, "io_stream", "name", 'VitisUnified')

config MONO model
-------------------------------------------------------
start running task : csynth
    with command: v++ -c --mode hls --config /media/tanawin/tanawin1701e/project7/vitisUnifiedTutorial/hls4mlprjBuildTest_io_stream_name_VitisUnified/hls_kernel_config.cfg --work_dir unifiedPrj
-------------------------------------------------------

****** v++ v2023.2 (64-bit)
  **** SW Build 4026344 on 2023-10-11-15:42:10
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2023 Advanced Micro Devices, Inc. All Rights Reserved.

Running Dispatch Server on port: 36051
INFO: [v++ 60-1548] Creating build summary session with primary output /media/tanawin/tanawin1701e/project7/vitisUnifiedTutorial/hls4mlprjBuildTest_io_stream_name_VitisUnified/unifiedWorkspace/myproject/unifiedPrj/unifiedPrj.hlscompile_summary, at Fri Aug 29 16:02:19 2025
INFO: [v++ 82-31] Launching vitis_hls: vitis_hls -nolog -run csynth -work_dir /media/tanawin/tanawin1701e/project7/vitisU